In [ ]:
!pip install transformers gradio sentencepiece nltk --quiet

     |████████████████████████████████| 5.5 MB 9.2 MB/s 
     |████████████████████████████████| 11.6 MB 47.5 MB/s 
     |████████████████████████████████| 1.3 MB 61.8 MB/s 
     |████████████████████████████████| 182 kB 85.1 MB/s 
     |████████████████████████████████| 7.6 MB 63.4 MB/s 
     |████████████████████████████████| 106 kB 82.2 MB/s 
     |████████████████████████████████| 278 kB 39.2 MB/s 
     |████████████████████████████████| 84 kB 4.8 MB/s 
     |████████████████████████████████| 56 kB 6.1 MB/s 
     |████████████████████████████████| 54 kB 3.9 MB/s 
     |████████████████████████████████| 84 kB 3.7 MB/s 
     |████████████████████████████████| 213 kB 85.5 MB/s 
     |████████████████████████████████| 55 kB 4.7 MB/s 
     |████████████████████████████████| 2.3 MB 56.5 MB/s 
     |████████████████████████████████| 64 kB 3.6 MB/s 
     |████████████████████████████████| 80 kB 12.1 MB/s 
     |████████████████████████████████| 68 kB 8.7 MB/s 
     |███████████████████████

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import nltk
nltk.download('punkt')
import gradio as gr
import pandas as pd
from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config
from transformers import AutoConfig, AutoModelWithLMHead, AutoTokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
#TITLE GENERATION
TITLE_MAX_LEN = 20

TITLE_MODEL_PATH = '/content/drive/MyDrive/nlp/title_generation_2.pt'
title_tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-common_gen")
title_tokenizer.pad_token = title_tokenizer.eos_token
title_config = AutoConfig.from_pretrained("mrm8488/t5-base-finetuned-common_gen")
title_model = AutoModelWithLMHead.from_pretrained(TITLE_MODEL_PATH, config = title_config)
title_model.to(device)
title_model.tie_weights()
title_model.eval()

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/auto/modeling_auto.py:1132: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
#ARTICLE GENERATION
bos = '<|startoftext|>'
eos = '<|EOS|>'
body = '<|body|>'
MAX_LEN = 200

MODEL_PATH = '/content/drive/MyDrive/nlp/nlp_2/article_generation_2.pt'
tokenizer = GPT2Tokenizer.from_pretrained("/content/drive/MyDrive/nlp/nlp_2/article_generation_token_2")

config = GPT2Config.from_json_file('/content/drive/MyDrive/nlp/nlp_2/config_2.json')

model = GPT2LMHeadModel.from_pretrained(MODEL_PATH, config=config)
#model.resize_token_embeddings(len(tokenizer))
model.to(device)
model.tie_weights()
model.eval()


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50261, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/nlp/test.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  119 non-null    int64 
 1   keywords    119 non-null    object
 2   Title       119 non-null    object
 3   Article     119 non-null    object
dtypes: int64(1), object(3)
memory usage: 3.8+ KB


In [ ]:
keywords = df['keywords'].to_list()

In [ ]:
article_list = []
title_list = []
inputs = title_tokenizer([keyword for keyword in keywords], return_tensors="pt", padding=True)
title_output_sequences = title_model.generate(
    input_ids=inputs["input_ids"].to(device, dtype=torch.long),
    attention_mask=inputs["attention_mask"].to(device, dtype=torch.long),
    do_sample=True,
    max_length = TITLE_MAX_LEN,
)

title = title_tokenizer.batch_decode(title_output_sequences, skip_special_tokens=True)
title_list.extend(title)
articles = {}
for i, headline in enumerate(title):
  print(i)
  headline_c = ' '.join([bos, headline,  body])
  input = tokenizer(headline_c, return_tensors="pt", padding=True)
  article = model.generate(
      input_ids=input["input_ids"].to(device, dtype=torch.long),
      attention_mask=input["attention_mask"].to(device, dtype=torch.long),
      do_sample=True,
      max_length = MAX_LEN,
  )
  decoded_article = tokenizer.decode(article[0], skip_special_tokens=True)
  articles[headline] = decoded_article.replace(headline, '')

for title, article in articles.items():
  article_list.append(article)

print(len(title))
print(len(article_list))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
69
119


In [ ]:
new_df = pd.DataFrame()

In [ ]:
new_df['keywords'] = keywords
new_df['title'] = title_list
new_df['article'] = article_list
new_df.to_csv('manual_label.csv')